
# AWS Lab Exercise — VPC Gateway Endpoint for S3 (Console Based)

### Objective
In this exercise, you will create a **VPC Gateway Endpoint for S3** to allow your **Private Subnet EC2 instance (EC2-B)** to access Amazon S3 without using the Internet Gateway.  
This ensures secure, private connectivity between your VPC and S3.

---

### Architecture Diagram
```
Region: ap-south-1 (Mumbai)
└── VPC (10.100.0.0/16)
    ├── Public Subnet (10.100.0.0/24) → EC2-A (Public + Private IP)
    ├── Private Subnet (10.100.11.0/24) → EC2-B (Private IP only)
    ├── Internet Gateway (attached to VPC)
    └── VPC Gateway Endpoint for S3 (connected to Private Subnet Route Table)
```
Private Subnet Route Table after endpoint creation:
- 10.100.0.0/16 → Local  
- `pl-xxxxxx` → `vpce-xxxxxx`

---

### High-Level Steps
1. Create a VPC, Internet Gateway, Public and Private Subnets  
2. Launch EC2 instances (EC2-A and EC2-B)  
3. Create an S3 bucket and upload a sample file  
4. Create an IAM role for EC2-B with S3 read-only permissions  
5. Try to access S3 from EC2-B (expected to fail)  
6. Create a **VPC Gateway Endpoint for S3** and update the Private Subnet Route Table  
7. Retry accessing S3 (should succeed this time)

---



## Step 1: Create VPC and Subnets

1. Navigate to **VPC Service** in AWS Console.  
2. Create a new VPC:  
   - **Name tag:** `VPC-A`  
   - **IPv4 CIDR block:** `10.100.0.0/16`  
   - **Tenancy:** Default  
   - Click **Create VPC**.  
3. Create two subnets under this VPC:  
   - **Public Subnet:** `VPC-A-Public`, CIDR `10.100.0.0/24`, Auto-assign public IP enabled.  
   - **Private Subnet:** `VPC-A-Private`, CIDR `10.100.11.0/24`, Auto-assign public IP disabled.  
4. Create a **Route Table** for each subnet:  
   - Public route table: add route `0.0.0.0/0` → Internet Gateway.  
   - Private route table: local route only.  
5. Associate each route table with the corresponding subnet.



## Step 2: Launch EC2 Instances

1. Launch **EC2-A** in the **Public Subnet**:
   - Public IP enabled.  
   - Security Group: allow **SSH (22)** from MyIP or anywhere.  

2. Launch **EC2-B** in the **Private Subnet**:
   - Public IP disabled.  
   - Security Group: allow **SSH (22)** and **ICMP (ping)** from `10.100.0.0/16`.  

3. Ensure connectivity between EC2-A and EC2-B as configured in the earlier exercises.



## Step 3: Create an S3 Bucket and Upload a Sample File

1. Open **Amazon S3** console → click **Create Bucket**.  
2. **Bucket name:** must be globally unique (e.g., `my-vpc-endpoint-lab-bucket`).  
3. Select **Region:** `ap-south-1`.  
4. Keep all defaults → click **Create bucket**.  
5. Open the created bucket → click **Upload** → **Add files** → select any sample file from your local machine → click **Upload**.  

This file will be accessed later from the private EC2 instance.



## Step 4: Create an IAM Role for EC2 to Access S3

1. Go to **IAM Console** → **Roles** → click **Create role**.  
2. Select **Use case:** EC2 → click **Next**.  
3. On the **Permissions** page, search for and select **AmazonS3ReadOnlyAccess** policy → click **Next**.  
4. **Role name:** `EC2_ROLE_FOR_S3_READONLY`.  
5. Click **Create role**.  
6. Once created, go to **EC2 Console** → select **EC2-B** → **Actions → Security → Modify IAM role** → choose `EC2_ROLE_FOR_S3_READONLY` → **Update IAM role**.

Now EC2-B has permission to read from S3 but still lacks network connectivity to reach S3.



## Step 5: Test S3 Access from EC2-B (Should Fail)

1. SSH into **EC2-A** from your local workstation using your `.pem` key file.  
   ```bash
   ssh -i your-key.pem ec2-user@<EC2-A-Public-IP>
   ```

2. From EC2-A, SSH into **EC2-B** (using private IP).  
   ```bash
   ssh -i key.pem ec2-user@10.100.11.x
   ```

3. On EC2-B, try to download the file from S3 using AWS CLI (make sure AWS CLI is installed).  
   ```bash
   aws s3 cp s3://<bucket-name>/<filename> /home/ec2-user/
   ```

> This command will **fail**, as there is currently **no internet route** from the private subnet to S3.



## Step 6: Create a VPC Gateway Endpoint for S3

1. Navigate to **VPC Console** → **Endpoints** → click **Create endpoint**.  
2. **Name tag:** `my-s3-endpoint`.  
3. **Service category:** AWS services.  
4. **Service name:** search for `com.amazonaws.ap-south-1.s3` (Type: Gateway).  
5. **VPC:** Select `VPC-A`.  
6. **Route tables:** Select the **Private Subnet Route Table**.  
7. Click **Create endpoint**.  

Once the endpoint is created, the route table for the private subnet will automatically include a new route for the S3 prefix list (`pl-xxxxxx`) pointing to the VPC endpoint (`vpce-xxxxxx`).



## Step 7: Test S3 Access Again (Should Succeed)

1. From **EC2-A**, SSH into **EC2-B** as before.  
2. Run the same S3 copy command:  
   ```bash
   aws s3 cp s3://<bucket-name>/<filename> /home/ec2-user/
   ```

3. This time, the file should successfully download.  

> ✅ **Explanation:** The private subnet can now reach S3 via the **VPC Gateway Endpoint**, which provides a direct, secure connection within the AWS network — no internet access required.



---

## Reflection

✅ Why did the initial download fail?  
→ The private subnet had no route to S3 or Internet Gateway.  

✅ How did the VPC Gateway Endpoint fix it?  
→ It provided a private route between VPC and S3 using AWS’s internal network.  

✅ Key Learning  
VPC Gateway Endpoints allow instances in private subnets to securely access S3 without using public internet routes, enhancing both security and compliance.

---
